In [1]:
import seaborn as sns

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep  7 16:02:02 2021

@author: epinsky
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Nov  5 14:37:29 2018

@author: epinsky
"""

# install yfinance version 0.1.62
#   !pip install yfinance==0.1.62
# run this  !pip install pandas_datareader
from pandas_datareader import data as web
import os
import pandas as pd
import yfinance as yf

def get_stock(ticker, start_date, end_date, s_window, l_window):
    try:
#       yf.pdr_override()
        df = yf.download(ticker, start=start_date, end=end_date)
# can use this as well        df = web.get_data_yahoo(ticker, start=start_date, end=end_date)
        df['Return'] = df['Adj Close'].pct_change()
        df['Return'].fillna(0, inplace = True)
        df['Date'] = df.index
        df['Date'] = pd.to_datetime(df['Date'])
        df['Month'] = df['Date'].dt.month
        df['Year'] = df['Date'].dt.year 
        df['Day'] = df['Date'].dt.day
        for col in ['Open', 'High', 'Low', 'Close', 'Adj Close']:
            df[col] = df[col].round(2)
        df['Weekday'] = df['Date'].dt.day_name()
        df['Week_Number'] = df['Date'].dt.strftime('%U')
        df['Year_Week'] = df['Date'].dt.strftime('%Y-%U')
        df['Short_MA'] = df['Adj Close'].rolling(window=s_window, min_periods=1).mean()
        df['Long_MA'] = df['Adj Close'].rolling(window=l_window, min_periods=1).mean()        
        col_list = ['Date', 'Year', 'Month', 'Day', 'Weekday', 
                    'Week_Number', 'Year_Week', 'Open', 
                    'High', 'Low', 'Close', 'Volume', 'Adj Close',
                    'Return', 'Short_MA', 'Long_MA']
        num_lines = len(df)
        df = df[col_list]
        print('read ', num_lines, ' lines of data for ticker: ' , ticker)
        return df
    except Exception as error:
        print(error)
        return None

try:
    ticker='SPY'
    input_dir = os.getcwd()
    output_file = os.path.join(input_dir, ticker + '.csv')
    df = get_stock(ticker, start_date='2018-09-01', end_date='2023-08-31', 
               s_window=14, l_window=50)
    df.to_csv(output_file, index=False)
    print('wrote ' + str(len(df)) + ' lines to file: ' + output_file)
except Exception as e:
    print(e)
    print('failed to get Yahoo stock data for ticker: ', ticker)

[*********************100%%**********************]  1 of 1 completed
read  1256  lines of data for ticker:  SPY
wrote 1256 lines to file: /Users/williammohr/BostonUniversity/CS677/Assignments/assignment_2/get_stock_data/hw_2_submission/SPY.csv


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep  7 16:02:02 2021

@author: epinsky
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Nov  5 14:37:29 2018

@author: epinsky
"""

# install yfinance version 0.1.62
#   !pip install yfinance==0.1.62
# run this  !pip install pandas_datareader
from pandas_datareader import data as web
import os
import pandas as pd
import yfinance as yf


try:
    ticker='GME'
    input_dir = os.getcwd()
    output_file = os.path.join(input_dir, ticker + '.csv')
    df = get_stock(ticker, start_date='2018-09-01', end_date='2023-08-31', 
               s_window=14, l_window=50)
    df.to_csv(output_file, index=False)
    print('wrote ' + str(len(df)) + ' lines to file: ' + output_file)
except Exception as e:
    print(e)
    print('failed to get Yahoo stock data for ticker: ', ticker)

[*********************100%%**********************]  1 of 1 completed
read  1256  lines of data for ticker:  GME
wrote 1256 lines to file: /Users/williammohr/BostonUniversity/CS677/Assignments/assignment_2/get_stock_data/hw_2_submission/GME.csv


##### Import stock data

In [10]:
ticker='GME'
input_dir = \
r'/Users/williammohr/BostonUniversity/CS677/assignment_2/get_stock_data/'
ticker_file = os.path.join(input_dir, ticker + '.csv')

try:   
    with open(ticker_file) as f:
        gme_lines = f.read().splitlines()
    print('opened file for ticker: ', ticker)
    """    your code for assignment 1 goes here
    """
    
except Exception as e:
    print(e)
    print('failed to read stock data for ticker: ', ticker)



[Errno 2] No such file or directory: '/Users/williammohr/BostonUniversity/CS677/assignment_2/get_stock_data/hw_2_submission/GME.csv'
failed to read stock data for ticker:  GME


In [11]:
spy_lines[0]

NameError: name 'spy_lines' is not defined

In [7]:
ticker='SPY'
input_dir = r'./get_stock_data/hw_2_submission'
ticker_file = os.path.join(input_dir, ticker + '.csv')

try:   
    with open(ticker_file) as f:
        spy_lines = f.read().splitlines()
    print('opened file for ticker: ', ticker)
    """    your code for assignment 1 goes here
    """
    
except Exception as e:
    print(e)
    print('failed to read stock data for ticker: ', ticker)



[Errno 2] No such file or directory: './get_stock_data/hw_2_submission/SPY.csv'
failed to read stock data for ticker:  SPY


1. if the Open price is higher than the ”Adj Close” price of yesterday (i.e. overnight return is positive) then you assume that the stock price will continue to increase during the day. Therefore, you buy $100 worth of shares at the ”Open” price and sell this number of shares at the end of the day (at adj close) to close your ”long” position. The number of shares will typically be fractional, like 1.12 or 0.89. Your profit/loss per share is (adj close - open).

2. if the open price is lower than the ”Adj Close” price of yesterday (i.e. overnigh return is negative) then you assume that the stock price will continue to fall during the day. Therefore, you sell short $100 worth of shares at the ”Open” price and buy this same number of shares at the end of the day (at adj close price) to close your ”short” position. You profit loss per share is (open - adj close)

3. unless the opening price equals yesterdays (adj) closing price, you will always day trade.  Assume in both cases (whether you establish a long or short position) that you have $100 to invest and you know the overnight returns. We ignore trading costs in our analysis.

4. for every day that you trade, you are given $100 (regardless of your previous trading performance).

5. finally, in your computation of percentages, number of shares, prices - round to 2 decimal points.

Plan: loop through data from the second day to the last day. Compare adj_close to open_price.  If open_price_t1 > adj_close_t0, append 100*(adj_close_t1-open_price_t1)/open_price_t1.  Likewise, if open_price_t0 > adj_close_t1, append 100*(open_price_t1-adj_close_t0)/(open_price_t1)

##### Initial Formatting

In [ ]:
def get_daily_profits(prices,threshold):
    """args --> str: ticker of stock; float: proportion of stock change that
    triggers a trade.
    return --> list: tuples of date, profit, and position"""
    daily_profits = \
    [(d[0],100/d[1]*(d[2]-d[1]),'long') 
     # profit is 100*(adj_close_t1 - open_t1)
     if (d[1]>(prices[i][2] + prices[i][2]*threshold)) # if open_t1 g.t. adj_close_t0
     else [(d[0],100/d[1]*(d[1]-d[2]),'short') 
           if ((prices[i][2]*(1-threshold))>d[1]) # if adj_close_t0 g.t. open_t1
           else (d[0], 0,'no_trade')][0]
     for i,d in enumerate(prices[1:])] 
        # begin trading on day2 based on d2 a.close
    daily_profits = [tuple([
    tuple([int(x) for x in 
           day[0].split(' ')[0].split('-')]), day[1], day[2]]) 
                     for day in daily_profits]
    return daily_profits

###### gme

1) Lines are initially a single string with comma separated values.  I need to convert each line to a list of values of the appropriate type.

In [ ]:
gme_lines[0]

In [ ]:
# Collect relevant indices
relev_inxs = [gme_lines[0].split(',').index(x) \
 for x in gme_lines[0].split(',')\
if x in ['Date','Open','Adj Close']
 ]

[0, 7, 12] is the list of relevant indices for momentum analysis purposes, corresponding to Date, Open, and Adj Close.

In [ ]:
gme_lines = [line.split(',') for line in gme_lines] # convert csv string to list

In [ ]:
# collect relev. header vals
headers = [header for inx,header in 
           enumerate(gme_lines.pop(0)) if inx in [0,7,12]]
headers = ['_'.join(header_val.split(' ')) for header_val in headers] # concat.

In [ ]:
gme_prices = [
    [val if inx==0 else float(val) # convert open, adj_close to floats
  for inx,val in enumerate(
      [val for inx,val in enumerate(line) # keep Date, Open, and Adj_
       if inx in relev_inxs])] 
    for line in gme_lines
]

In [ ]:
gme_daily_profits = get_daily_profits(gme_prices,0)

###### spy

1) Lines are initially a single string with comma separated values.  I need to convert each line to a list of values of the appropriate type.

In [ ]:
spy_lines[0]

In [ ]:
# Collect relevant indices
relev_inxs = [spy_lines[0].split(',').index(x) \
 for x in spy_lines[0].split(',')\
if x in ['Date','Open','Adj Close']
 ]

[0, 7, 12] is the list of relevant indices for momentum analysis purposes, corresponding to Date, Open, and Adj Close.

In [ ]:
spy_lines = [line.split(',') for line in spy_lines] # convert csv string to list

In [ ]:
# collect relev. header vals
headers = [header for inx,header in 
           enumerate(spy_lines.pop(0)) if inx in [0,7,12]]
headers = ['_'.join(header_val.split(' ')) for header_val in headers] # concat.

In [ ]:
spy_prices = [
    [val if inx==0 else float(val) # convert open, adj_close to floats
  for inx,val in enumerate(
      [val for inx,val in enumerate(line) # keep Date, Open, and Adj_
       if inx in relev_inxs])] 
    for line in spy_lines
]

In [ ]:
spy_daily_profits = get_daily_profits(spy_prices, 0)

##### Profit Analysis

For a particular time period (e.g. year): 

Statistic         | Definition
------------------|--------------------------------------------------------
 \|S\|            | number of days when a short position was taken
 \|L\|            | number of days when a long position was taken
 \|S+L\|          | number of days when either a long or short position was taken
 P(S)             | total profit/loss for 'short' days
 P(S) / \|S\|     | avg daily profit for 'short position' days
 P(L) / \|L\|     | avg daily profit for 'long position' days
 P(S+L) / \|S+L\| | avg daily profit for trading days

###### Trading statistics functions

In [ ]:
def short_days(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: integer |S|"""
    return sum([1 for day in time_period if day[2] == 'short'])

In [ ]:
def short_profit(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: float P(S) rounded to 2 decimal places"""
    return round(sum([day[1] for day in time_period if day[2] == 'short']),2)

In [ ]:
def short_profit_mean(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: float P(S)/|S| rounded to 2 decimal places"""
    return round(short_profit(time_period)/short_days(time_period),2)

In [ ]:
def long_days(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: integer |L|"""
    return sum([1 for day in time_period if day[2] == 'long'])

In [ ]:
def long_profit(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: float P(L) rounded to 2 decimal places"""
    return round(sum([day[1] for day in time_period if day[2] == 'long']),2)

In [ ]:
def long_profit_mean(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: float P(L)/|L| rounded to 2 decimal places"""
    return round(long_profit(time_period)/long_days(time_period),2)

In [ ]:
def trading_days(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: integer |S+L|"""
    return sum([1 for day in time_period if day[2] in ('long','short')])

In [ ]:
def total_profit(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: float P(S+L) rounded to 2 decimal places"""
    return round(sum([day[1] for day in time_period]),2)

In [ ]:
def total_profit_mean(time_period):
    """arg: a list of profit data-tuples, i.e. (date,profit,position)
    return: float P(S+L)/|S+L| rounded to 2 decimal places"""
    return round(total_profit(time_period)/trading_days(time_period),2)

In [ ]:
b = ['a']

In [ ]:
b.append(['c'])

In [ ]:
def trading_stats(trading_periods):
    """arg: a list of lists of tuples containing trading data (date,profit,position)
    return: a year-by-year list of (|S|, P(S), P(S)/|S|, |L|, P(L), P(L)/|L|,
    |S+L|, P(S+L), P(S+L)/|S+L|"""
    stats = []
    for p in trading_periods:
        try:
            spm = short_profit_mean(p)
        except ZeroDivisionError:
            spm = 0
        try:
            lpm = long_profit_mean(p)
        except ZeroDivisionError:
            lpm = 0           
        stats = stats + [(short_days(p), short_profit(p),
               spm, long_days(p), long_profit(p), lpm, trading_days(p),
               total_profit(p), total_profit_mean(p))]
        
    return stats

##### Split profit data into yearly intervals

I'll use the property that for every date d0 with year d0_y, month d0_m, day d0_d  prior to any other date with yyyy, mm, dd being respectively d1_y, d1_m, d1_d the following holds:

d1_y*2080 + d1_m*32 + d1_d > d0_y*2080 + d0_m*32 + d0_d

Thus, I can pop the first day's data while (d1_y*2080 + d1_m*32 + d1_d) < (d0_y*2080 + d0_m*32 + d0_d + (d1_y-d0_y)*2080)

In [ ]:
def date_transform(date):
    """arg: date as tuple of ints (YYYY,MM,DD) 
    return: date transformed into int by multiplying year by 2080, month by 32,
    day by 1, and summing"""
    transform_factors = [2080,40,1]
    return sum([i*j for i,j in zip(date,transform_factors)])

In [ ]:
def five_years(original_trading_data,ticker):
    """arg: 5 years of data; ticker string, i.e. 'SPY'
    return: list of data chopped into yearly periods"""
    
    evaporating_data = [td for td in original_trading_data]
    ticker = ticker.lower()
    years = [1,2,3,4]
    d0 = date_transform(original_trading_data[0][0])
    d0_transformed = date_transform(evaporating_data[0][0])
    for year in years:
        placeholder = f'{ticker}_{year}'
        globals()[placeholder] = [evaporating_data.pop(0) for _ in 
                                 evaporating_data 
                                if  date_transform(evaporating_data[0][0]) < 
                                 d0+year*2080]
    placeholder = f'{ticker}_5'
    globals()[placeholder]=evaporating_data
    placeholder = f'{ticker}_1_5'
    globals()[placeholder] = [td for td in original_trading_data] # all years    
    # split_yearly = eval(f'[{ticker}_{year} for year in years]')
    print(f'{ticker}_year, i.e {ticker}_1, {ticker}_2, ... {ticker}_5,\
 {ticker}_1_5 are now in the local namespace.')

###### GME

In [ ]:
five_years(gme_daily_profits,'gme')

verify that the year separations are correct

In [ ]:
print('{},{},{},{},{},{},{},{},{},{},{},{}'.format(gme_1[0][0],gme_1[-1][0],
                                    gme_2[0][0],gme_2[-1][0],gme_3[0][0],
                                    gme_3[-1][0],gme_4[0][0],gme_4[-1][0],
                                    gme_5[0][0],gme_5[-1][0],gme_1_5[0][0], 
                                    gme_1_5[-1][0]
                                   ))

In [ ]:
len(gme_daily_profits)

In [ ]:
sum([len(p) for p in [gme_1,gme_2,gme_3,gme_4,gme_5]])

In [ ]:
gme_5

No missing vals. Hurray!

In [ ]:
[len(p) for p in [gme_1,gme_2,gme_3,gme_4,gme_5,gme_1_5]]

~ 252 trading days per year.  Hurray!

###### SPY

In [ ]:
five_years(spy_daily_profits,'spy')

verify that the year separations are correct

In [ ]:
len(spy_daily_profits)

In [ ]:
print('{},{},{},{},{},{},{},{},{},{}'.format(spy_1[0][0],spy_1[-1][0],
                                    spy_2[0][0],spy_2[-1][0],spy_3[0][0],
                                    spy_3[-1][0],spy_4[0][0],spy_4[-1][0],
                                    spy_5[0][0],spy_5[-1][0]
                                   ))

In [ ]:
[len(p) for p in [spy_1,spy_2,spy_3,spy_4,spy_5]]

~252 trades per day.  Hurray!

No missing vals. Hurray!

In [ ]:
print('{},{},{},{},{},{},{},{},{},{},{},{}'.format(spy_1[0][0],spy_1[-1][0],
                                    spy_2[0][0],spy_2[-1][0],spy_3[0][0],
                                    spy_3[-1][0],spy_4[0][0],spy_4[-1][0],
                                    spy_5[0][0],spy_5[-1][0],spy_1_5[0][0], 
                                    spy_1_5[-1][0]
                                   ))

In [ ]:
sum([len(p) for p in [spy_1,spy_2,spy_3,spy_4,spy_5]])

No missing vals. Hurray!

In [ ]:
[len(p) for p in [spy_1,spy_2,spy_3,spy_4,spy_5,spy_1_5]]

~ 252 trading days per year.  Hurray!

##### stock metrics

In [ ]:
def markdown_table(stats):
    headers = ['Year', '|S|', 'P(S)', 'P(S)/|S|',
               '|L|', 'P(L)','P(L)/|L|','|S+L|', 'P(S+L)', 'P(S+L)/|S+L|']
    w_year = [val for (y,s) in 
        zip(['1','2','3','4','5','1_5'], [list(tup) for tup in stats]) for val in (y,s)]
    w_year_all_list = [[val] if type(val) != list else val for val in w_year]
    w_year_flat = [val for val_list in w_year_all_list for val in val_list]
    flattened_stats = headers + [v if type(v)==str else str(v) for v in w_year_flat]
    print("\n\
    {:5s} | {:4s} | {:8s} | {:8s} | {:4s} | {:8s} | {:8s} | {:5s} | {:6s} | {:12s}\n\
    ------------------------------------------------------------------------------------------------\n\
    {:5s} | {:4s} | {:8s} | {:8s} | {:4s} | {:8s} | {:8s} | {:5s} | {:6s} | {:12s}\n\
    {:5s} | {:4s} | {:8s} | {:8s} | {:4s} | {:8s} | {:8s} | {:5s} | {:6s} | {:12s}\n\
    {:5s} | {:4s} | {:8s} | {:8s} | {:4s} | {:8s} | {:8s} | {:5s} | {:6s} | {:12s}\n\
    {:5s} | {:4s} | {:8s} | {:8s} | {:4s} | {:8s} | {:8s} | {:5s} | {:6s} | {:12s}\n\
    {:5s} | {:4s} | {:8s} | {:8s} | {:4s} | {:8s} | {:8s} | {:5s} | {:6s} | {:12s}\n\
    {:5s} | {:4s} | {:8s} | {:8s} | {:4s} | {:8s} | {:8s} | {:5s} | {:6s} | {:12s}".format(*flattened_stats))

In [ ]:
trading_stats([gme_daily_profits])

In [ ]:
trading_stats([spy_daily_profits])

###### GME Statistics

In [ ]:
markdown_table(trading_stats([gme_1,gme_2,gme_3,gme_4,gme_5,gme_1_5]))

###### SPY Statistics

In [ ]:
markdown_table(trading_stats([spy_1,spy_2,spy_3,spy_4,spy_5,spy_1_5]))

##### With restriction

GME

In [ ]:
h_p_gme = []
for i in range(0,100):
    h_p = i/1000
    h_p_gme = h_p_gme + [(h_p, trading_stats([get_daily_profits(gme_prices,h_p)])[-1][-1])]

SPY

In [ ]:
h_p_spy = []
for i in range(0,100):
    h_p = i/1000
    h_p_spy = h_p_spy + [(h_p, trading_stats([get_daily_profits(spy_prices,h_p)])[-1][-1])]

Zipped

In [ ]:
[val for val in zip(h_p_gme, h_p_spy)]

##### Long and short

In [ ]:
h_p_gme_long = []
for i in range(0,100):
    h_p = i/1000
    h_p_gme_long = h_p_gme_long + [(h_p, trading_stats([get_daily_profits(gme_prices,h_p)])[-1][5])]

In [ ]:
h_p_spy_long = []
for i in range(0,100):
    h_p = i/1000
    h_p_spy_long = h_p_spy_long + [(h_p, trading_stats([get_daily_profits(spy_prices,h_p)])[-1][5])]

In [ ]:
[val for val in zip(h_p_gme_long,h_p_spy_long)]

In [ ]:
h_p_gme_short = []
for i in range(0,100):
    h_p = i/1000
    h_p_gme_short = h_p_gme_short + [(h_p, trading_stats([get_daily_profits(gme_prices,h_p)])[-1][5])]

In [ ]:
h_p_spy_short = []
for i in range(0,100):
    h_p = i/1000
    h_p_spy_short = h_p_spy_short + [(h_p, trading_stats([get_daily_profits(spy_prices,h_p)])[-1][5])]

In [ ]:
[val for val in zip(h_p_gme_short,h_p_spy_short)]

In [ ]:
'abcd'[-1]

##### Last digit analysis

In [ ]:
def last_digit(prices):
    return [int(str(d[1])[-1]) for d in prices]

In [ ]:
def digit_counts(digits):
    zeros = 0; ones = 0; twos = 0; threes = 0; fours = 0;
    fives = 0; sixes = 0; sevens = 0; eights = 0; nines = 0;
    for d in digits:
        if d == 0:
            zeros += 1
        elif d == 1:
            ones += 1
        elif d == 2:
            twos += 1
        elif d == 3:
            threes += 1
        elif d == 4:
            fours += 1
        elif d == 5:
            fives += 1
        elif d == 6:
            sixes += 1
        elif d == 7:
            sevens += 1
        elif d == 8:
            eights += 1
        elif d == 9:
            nines += 1
    return([zeros,ones,twos,threes,fours,fives,sixes,sevens,eights,nines])

In [ ]:
gme_digit_counts = digit_counts(last_digit(gme_prices))
gme_digit_counts

In [ ]:
sns.barplot(x = ['zeros','ones','twos','threes','fours','fives','sixes','sevens',
                 'eights','nines'],
    y=[51, 127, 132, 123, 129, 223, 107, 115, 125, 124])

In [ ]:
spy_digit_counts = digit_counts(last_digit(spy_prices))
spy_digit_counts

In [ ]:
import seaborn as sns

In [ ]:
sns.barplot(x = ['zeros','ones','twos','threes','fours','fives','sixes','sevens',
                 'eights','nines'],
    y=[13, 147, 154, 140, 145, 127, 122, 146, 126, 136])

In [ ]:
sns.barplot(x = ['zeros','ones','twos','threes','fours','fives','sixes','sevens',
                 'eights','nines'],
    y=[13, 147, 154, 140, 145, 127, 122, 146, 126, 136])

In [ ]:
print('all code runs!')